In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from model import predict # Функция, позволяет получить предсказание нейронки.
from check_budget import check_budget # функция проверки бюджета. Проверяйте допустимость решения до сабмита

In [2]:
bins_path = "nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)
BUDGET = 10 # разрешенное количество изменений транзакций для каждого пользователя

output_path = "train_changed.csv" # куда сохранить атакованные транзакции
transactions_path = "../../transactions_finetune.csv"    # путь до файла с транзакциями, которые атакуются

In [22]:
# у нас нет разметки для тех транзакций, которые мы атакуем - но у нас есть модель.
# Давайте посчитаем вероятность того, что пользователь принадлежит к классу 1
result = predict(transactions_path, bins_path, model_path, random_seed=20230206)
result.head()

Global seed set to 20230206


,user_id,target
0,69,0.014768
1,140,0.140376
2,196,0.039746
3,400,0.009377
4,544,0.048174


In [5]:
result["target"].describe()

count    7080.000000
mean        0.043170
std         0.052960
min         0.000331
25%         0.009206
50%         0.022457
75%         0.054486
max         0.424275
Name: target, dtype: float64

In [6]:
# давайте в качестве порога использовать середину этого диапазона
# все что выше - пусть будет предсказано 1, что ниже - 0
threshold = result.target.median()

In [7]:
# Найдем пользователя, для которого нейронка предсказала самое большое значение таргета.
# Это будет наш Герой, Образцовый Положительный Пользователь
hero_user = result.user_id.loc[result.target.argmax()]
hero_user

469439

In [8]:
# Найдем так пользователя с самым низким таргетом
# Это будет наш Неудачник
poor_user = result.user_id.loc[result.target.argmin()]
poor_user

753340

In [11]:
# границы допустимых решений.

with open(quantiles_path, 'r') as f:
    quantiles = json.load(f)

In [12]:
# для каждого кода заданы лимиты положительных и отрифательных значений
# Вот, например, диапазон, в котором должны лежать суммы для ьcc_code 4111
quantiles["positive"]["min"]["4111"], quantiles["positive"]["max"]["4111"]

(8.145073890686035, 45117.69001)

In [13]:
# Читаем файл с исходными транзакциями
df_transactions = (
    pd.read_csv(
        transactions_path,
        parse_dates=["transaction_dttm"],
        dtype={"user_id": int, "mcc_code": int, "currency_rk": int, "transaction_amt": float},
    )
)

In [14]:
random.seed(20230206)

#threshold = pd.read_csv("../data/target_finetune.csv").target.mean()  # вероятность, по которой мы считаем таргет

df_transactions = pd.read_csv(
    transactions_path,
    parse_dates=["transaction_dttm"],
    dtype={"user_id": int, "mcc_code": int, "currency_rk": int, "transaction_amt": float},
)

bins_path = "nn_bins.pickle"
model_path = "nn_weights.ckpt"
target = predict(transactions_path, bins_path, model_path)


one_idx = target.index[target.target > threshold]  # Эти пользователи похожи на Героя
zero_idx = target.index[target.target <= threshold] # А эти на Неудачника

users = target.user_id.values

one_users = users[one_idx]
zero_users = users[zero_idx]

for user in tqdm(users):
    if user in one_users:
        copy_from = poor_user # похожим на Героя скопируем 10 последних транзакций Неудачника
    else:
        copy_from = hero_user # А похожим на Неудачника наоборот

    idx_to = df_transactions.index[df_transactions.user_id == user][-BUDGET // 2:]
    idx_from = df_transactions.index[df_transactions.user_id == copy_from][-BUDGET // 2:]
    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)
    df_transactions.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values
    
    idx_to = df_transactions.index[df_transactions.user_id == user][:BUDGET // 2]
    idx_from = df_transactions.index[df_transactions.user_id == copy_from][:BUDGET // 2]
    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)
    df_transactions.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values
df_transactions.to_csv(output_path, index=False)


Global seed set to 20230206
100%|██████████| 7080/7080 [00:33<00:00, 214.09it/s]


In [15]:
check_budget(transactions_path, output_path, quantiles_path) # Не забываем проверять бюджет перед самбитом!

100%|██████████| 2124000/2124000 [00:41<00:00, 51520.34it/s]


True